In [104]:
import pandas as pd
import os

In [105]:
def modify_df(file_name):
    """
    A function that reads file in csv format which was exported by LabelStudio.
    This function will transform given csv into dataframe that is more suitable for our sound classification.
    :param file_name: the string object that represents file name
    :returns: pandas.DataFrame object that represents data from the csv turned into our style.
    """
    data = pd.read_csv(file_name)
    
    file_names = list()
    start = list()
    end = list()
    class_name = list()

    for i in range(len(data)):
        instance = data.loc[i]
        cur_file_name = os.path.join(os.getcwd(), instance['audio'][1:])
        cur_class = eval(instance['intent'])  # Each values are recorded as str, so need eval
        cur_time_lines = eval(data.loc[i]['labels'])

        for j in range(len(cur_class)):
            cur_start = float(cur_time_lines[j]['start'])
            cur_end = float(cur_time_lines[j]['end'])

            start.append(cur_start)
            end.append(cur_end)
            class_name.append(cur_class[j])
            file_names.append(cur_file_name)

    modified_df = pd.DataFrame()
    modified_df['file_name'] = file_names
    modified_df['start'] = start
    modified_df['end'] = end
    modified_df['class'] = class_name
    
    return modified_df

In [113]:
from PIL import Image
from pydub import AudioSegment
import sqlite3
import string
import os

def generate_db():
    """
    A function that generates training_data/data.db file and initializes the table values.
    :return: None
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    conn.execute("CREATE TABLE data(data_name TEXT, data_count INTEGER)")
    
    default_data = [(x, 0) for x in string.ascii_lowercase]
    default_data = default_data + [(str(x), 0) for x in range(0, 10, 1)]
    default_data.append(("null", 0))
    
    cur.executemany("INSERT INTO data VALUES(?, ?)", default_data)
    
    conn.commit()
    conn.close()
    
def get_data_count(data_name):
    """
    A function that gets data count from data.db for specific data_name
    :param data_name: the data_name to look for data counts
    :return: returns count of data_count value for provided data_name, returns -1 if data_name does not exist.
    """
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("SELECT data_count FROM data WHERE data_name=\"" + data_name + "\"")
    rows = cur.fetchall()
    try:
        return rows[0][0]
    except IndexError:
        return -1
    
    conn.close()
    
def update_data_count(data_name):
    """
    A function that adds 1 to the current data_count for specified data_name
    :param data_name: the data_name to update values into
    :return: None
    """
    before_count = get_data_count(data_name)    
    new_count = before_count + 1
    
    conn = sqlite3.connect("./training_data/data.db")
    cur = conn.cursor()
    cur.execute("UPDATE data SET data_count=" + str(new_count) + " WHERE data_name=\"" + data_name + "\"")
    conn.commit()
    conn.close()

def generate_all_directories():
    """
    A function that generates training_data directories.
    This will generate trainig_data/a~z and 0~9 directories.
    :return: None
    """
    dir_names = [x for x in string.ascii_lowercase]
    dir_names = dir_names + [str(x) for x in range(0, 10, 1)]

    training_data_path = os.path.join(os.getcwd(), "training_data")

    if not os.path.exists(training_data_path):  # generate training_data directory
        os.mkdir(training_data_path)
    
    for i in dir_names:  # generate subdirectories such as a ~ z, 0 ~ 9
        new_dir= os.path.join(training_data_path, i)
        if not os.path.exists(new_dir):
            os.mkdir(new_dir)

def process_df(df):
    """
    A function that splits each sound classifications from dataframe into different files
    :param df: the pandas.DataFrame object that represents single row of information.
    """
    file_name = df['file_name']
    new_audio = AudioSegment.from_wav(file_name)
    
    start = df['start'] * 1000
    end = df['end'] * 1000
    class_name = df['class']
    
    tmp_audio = new_audio[start:end]
    
    save_dir = os.path.join(os.getcwd(), "training_data", class_name)
    new_file_name = str(get_data_count(class_name)) + ".wav"
    new_file_name = os.path.join(save_dir, new_file_name)
    update_data_count(class_name)
    tmp_audio.export(new_file_name, format="wav")

In [107]:
df = modify_df("project-3-at-2022-06-22-10-47-a1bccc9f.csv")

In [108]:
df

,file_name,start,end,class
0,/home/gooday2die/projects/Anti-Captcha/sounddo...,0.006369,0.630562,y
1,/home/gooday2die/projects/Anti-Captcha/sounddo...,1.082783,1.585958,r
2,/home/gooday2die/projects/Anti-Captcha/sounddo...,1.796146,2.337537,9
3,/home/gooday2die/projects/Anti-Captcha/sounddo...,2.872559,3.388473,k
4,/home/gooday2die/projects/Anti-Captcha/sounddo...,3.904388,4.350239,c
...,...,...,...,...
492,/home/gooday2die/projects/Anti-Captcha/sounddo...,0.019549,0.560398,r
493,/home/gooday2die/projects/Anti-Captcha/sounddo...,1.251121,1.883198,h
494,/home/gooday2die/projects/Anti-Captcha/sounddo...,2.515274,2.906250,4
495,/home/gooday2die/projects/Anti-Captcha/sounddo...,3.440583,3.948850,9


In [115]:
try:
    generate_db()
    print("[+] DB not found, generating a new one")
except sqlite3.OperationalError:  # when db already exists
    pass
generate_all_directories()

for i in range(len(df)):
    print("[+] Processing file " + str(i + 1) + " / " + str(len(df)))
    process_df(df.loc[i])

[+] DB not found, generating a new one
[+] Processing file 0 / 497
[+] Processing file 1 / 497
[+] Processing file 2 / 497
[+] Processing file 3 / 497
[+] Processing file 4 / 497
[+] Processing file 5 / 497
[+] Processing file 6 / 497
[+] Processing file 7 / 497
[+] Processing file 8 / 497
[+] Processing file 9 / 497
[+] Processing file 10 / 497
[+] Processing file 11 / 497
[+] Processing file 12 / 497
[+] Processing file 13 / 497
[+] Processing file 14 / 497
[+] Processing file 15 / 497
[+] Processing file 16 / 497
[+] Processing file 17 / 497
[+] Processing file 18 / 497
[+] Processing file 19 / 497
[+] Processing file 20 / 497
[+] Processing file 21 / 497
[+] Processing file 22 / 497
[+] Processing file 23 / 497
[+] Processing file 24 / 497
[+] Processing file 25 / 497
[+] Processing file 26 / 497
[+] Processing file 27 / 497
[+] Processing file 28 / 497
[+] Processing file 29 / 497
[+] Processing file 30 / 497
[+] Processing file 31 / 497
[+] Processing file 32 / 497
[+] Processing

[+] Processing file 284 / 497
[+] Processing file 285 / 497
[+] Processing file 286 / 497
[+] Processing file 287 / 497
[+] Processing file 288 / 497
[+] Processing file 289 / 497
[+] Processing file 290 / 497
[+] Processing file 291 / 497
[+] Processing file 292 / 497
[+] Processing file 293 / 497
[+] Processing file 294 / 497
[+] Processing file 295 / 497
[+] Processing file 296 / 497
[+] Processing file 297 / 497
[+] Processing file 298 / 497
[+] Processing file 299 / 497
[+] Processing file 300 / 497
[+] Processing file 301 / 497
[+] Processing file 302 / 497
[+] Processing file 303 / 497
[+] Processing file 304 / 497
[+] Processing file 305 / 497
[+] Processing file 306 / 497
[+] Processing file 307 / 497
[+] Processing file 308 / 497
[+] Processing file 309 / 497
[+] Processing file 310 / 497
[+] Processing file 311 / 497
[+] Processing file 312 / 497
[+] Processing file 313 / 497
[+] Processing file 314 / 497
[+] Processing file 315 / 497
[+] Processing file 316 / 497
[+] Proces

In [116]:
from IPython.display import Audio 
Audio("/home/gooday2die/projects/Anti-Captcha/sounddooo/sounds/training_data/j/0.wav", autoplay=True)